In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from tensorflow.keras import  models, layers

IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 50

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        "/content/drive/MyDrive/Datastets/Splitted/train",
        shuffle = True,
        image_size = (IMAGE_SIZE, IMAGE_SIZE),
        batch_size = BATCH_SIZE
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        "/content/drive/MyDrive/Datastets/Splitted/val",
        shuffle = True,
        image_size = (IMAGE_SIZE, IMAGE_SIZE),
        batch_size = BATCH_SIZE
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
        "/content/drive/MyDrive/Datastets/Splitted/test",
        shuffle = True,
        image_size = (IMAGE_SIZE, IMAGE_SIZE),
        batch_size = BATCH_SIZE
)

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

Found 32083 files belonging to 33 classes.
Found 4002 files belonging to 33 classes.
Found 4000 files belonging to 33 classes.


## VGG16


In [ ]:
def create_VGG16(input_shape, n_classes, optimizer, fine_tune=0):
    """
    Compiles a VGG16 model with pre-trained layers.
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training.
    fine_tune: int - The number of pre-trained layers to unfreeze. If set to 0, all
                     pre-trained layers will freeze during training
    """

    model_base = tf.keras.applications.VGG16(include_top=False, # exclude the model's fully-connected layers
                     weights='imagenet', # weights pre-trained using the Imagenet
                     input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    if fine_tune > 0:
        for layer in model_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in model_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model (i.e. 3 fully-connected layers).
    inputs = tf.keras.Input(shape=(input_shape))
    top_model = tf.keras.applications.vgg16.preprocess_input(inputs)
    top_model = model_base(top_model)
    top_model = layers.Flatten(name="flatten")(top_model)
    top_model = layers.Dense(4096, activation='relu')(top_model)
    top_model = layers.Dense(4096, activation='relu')(top_model)
    top_model = layers.Dense(4096, activation='relu')(top_model)
    output_layer = layers.Dense(n_classes, activation='softmax')(top_model)
    
    # Create a final output.
    model = models.Model(inputs=inputs, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    
    return model



VGG16_model = create_VGG16((IMAGE_SIZE, IMAGE_SIZE, 3), 33, 
                           optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=10**(-7)),
                           fine_tune=2)

history_logger = CSVLogger('VGG16_model_Adam.csv', separator=",", append=True)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

history = VGG16_model.fit(train_ds,
                    epochs=EPOCHS,
                    validation_data=val_ds,
                    callbacks=[history_logger, earlyStopping])

VGG16_model.save('VGG16_model_Adam.keras')

Epoch 1/50


## ResNet50

In [ ]:
def create_ResNet50(input_shape, n_classes, optimizer, fine_tune=0):
    """
    Compiles a ResNet50 model with pre-trained layers.
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training.
    fine_tune: int - The number of pre-trained layers to unfreeze. If set to 0, all
                     pre-trained layers will freeze during training
    """

    model_base = tf.keras.applications.ResNet50(include_top=False, # exclude the model's fully-connected layers
                                             weights='imagenet', # weights pre-trained using the Imagenet
                                             input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    if fine_tune > 0:
        for layer in model_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in model_base.layers:
            layer.trainable = False

    # Preprocessing layers
    resize_and_rescale = tf.keras.Sequential([
                                  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
                                  layers.experimental.preprocessing.Rescaling(1.0/255)])

    # Create a new 'top' of the model.
    inputs = tf.keras.Input(shape=(input_shape))
    top_model = resize_and_rescale(inputs)
    top_model = model_base(top_model)
    top_model = layers.Flatten(name="flatten")(top_model)
    output_layer = layers.Dense(n_classes, activation='softmax')(top_model)
    
    # Create a final output.
    model = models.Model(inputs=inputs, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    
    return model



ResNet50_model = create_ResNet50((IMAGE_SIZE, IMAGE_SIZE, 3), 33, 
                                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=10**(-7)),
                                 fine_tune=4)

history_logger = CSVLogger('ResNet50_model_Adam.csv', separator=",", append=True)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

history = ResNet50_model.fit(train_ds,
                    epochs=EPOCHS,
                    validation_data=val_ds,
                    callbacks=[history_logger, earlyStopping])

ResNet50_model.save('ResNet50_model_Adam.keras')

94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/50


## Inception-V3

In [ ]:
def create_InceptionV3(input_shape, n_classes, optimizer, fine_tune=0):
    """
    Compiles a Inception-V3 model with pre-trained layers.
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training.
    fine_tune: int - The number of pre-trained layers to unfreeze. If set to 0, all
                     pre-trained layers will freeze during training
    """

    model_base = tf.keras.applications.InceptionV3(include_top=False, # exclude the model's fully-connected layers
                                             weights='imagenet', # weights pre-trained using the Imagenet
                                             input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    if fine_tune > 0:
        for layer in model_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in model_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model.
    inputs = tf.keras.Input(shape=(input_shape))
    top_model = tf.keras.applications.inception_v3.preprocess_input(inputs)
    top_model = model_base(top_model)
    top_model = layers.Flatten(name="flatten")(top_model)
    output_layer = layers.Dense(n_classes, activation='softmax')(top_model)
    
    # Create a final output.
    model = models.Model(inputs=inputs, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    
    return model



InceptionV3_model = create_InceptionV3((IMAGE_SIZE, IMAGE_SIZE, 3), 33, 
                                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=10**(-7)),
                                 fine_tune=1)

history_logger = CSVLogger('InceptionV3_model_Adam.csv', separator=",", append=True)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

history = InceptionV3_model.fit(train_ds,
                    epochs=EPOCHS,
                    validation_data=val_ds,
                    callbacks=[history_logger, earlyStopping])

InceptionV3_model.save('InceptionV3_model_Adam.keras')

87910968/87910968 [==============================] - 0s 0us/step
Epoch 1/50


## Xception

In [ ]:
def create_Xception(input_shape, n_classes, optimizer, fine_tune=0):
    """
    Compiles a Xception model with pre-trained layers.
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training.
    fine_tune: int - The number of pre-trained layers to unfreeze. If set to 0, all
                     pre-trained layers will freeze during training
    """

    model_base = tf.keras.applications.Xception(include_top=False, # exclude the model's fully-connected layers
                                             weights='imagenet', # weights pre-trained using the Imagenet
                                             input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    if fine_tune > 0:
        for layer in model_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in model_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model.
    inputs = tf.keras.Input(shape=(input_shape))
    top_model = tf.keras.applications.xception.preprocess_input(inputs)
    top_model = model_base(top_model)
    top_model = layers.Flatten(name="flatten")(top_model)
    output_layer = layers.Dense(n_classes, activation='softmax')(top_model)
    
    # Create a final output.
    model = models.Model(inputs=inputs, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    
    return model



Xception_model = create_Xception((IMAGE_SIZE, IMAGE_SIZE, 3), 33, 
                                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=10**(-7)),
                                 fine_tune=2)

history_logger = CSVLogger('Xception_model_Adam.csv', separator=",", append=True)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

history = Xception_model.fit(train_ds,
                    epochs=EPOCHS,
                    validation_data=val_ds,
                    callbacks=[history_logger, earlyStopping])

Xception_model.save('Xception_model_Adam.keras')

83683744/83683744 [==============================] - 1s 0us/step
Epoch 1/50


## DenseNet201

In [ ]:
def create_DenseNet201(input_shape, n_classes, optimizer, fine_tune=0):
    """
    Compiles a DenseNet201 model with pre-trained layers.
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training.
    fine_tune: int - The number of pre-trained layers to unfreeze. If set to 0, all
                     pre-trained layers will freeze during training
    """

    model_base = tf.keras.applications.DenseNet201(include_top=False, # exclude the model's fully-connected layers
                                             weights='imagenet', # weights pre-trained using the Imagenet
                                             input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    if fine_tune > 0:
        for layer in model_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in model_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model.
    inputs = tf.keras.Input(shape=(input_shape))
    top_model = tf.keras.applications.densenet.preprocess_input(inputs)
    top_model = model_base(top_model)
    top_model = layers.Flatten(name="flatten")(top_model)
    top_model = layers.Dense(256, activation='relu')(top_model)
    output_layer = layers.Dense(n_classes, activation='softmax')(top_model)
    
    # Create a final output.
    model = models.Model(inputs=inputs, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    
    return model



DenseNet201_model = create_DenseNet201((IMAGE_SIZE, IMAGE_SIZE, 3), 33, 
                                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=10**(-7)),
                                 fine_tune=3)

history_logger = CSVLogger('DenseNet201_model_Adam.csv', separator=",", append=True)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

history = DenseNet201_model.fit(train_ds,
                    epochs=EPOCHS,
                    validation_data=val_ds,
                    callbacks=[history_logger, earlyStopping])

DenseNet201_model.save('DenseNet201_model_Adam.keras')

Epoch 1/50
